# Clean Data of the results of experiment 2
- This program is used to export the txt file to dataframe in python. The txt is captured from the experiment#2 in Github. File location : https://github.com/tawanmeow/privacy-sdn-logfile/tree/main/Experiment2
- Find maximum throughput by sending packets from Edge4 to Edge1 and Edge5 simultaneously.
### Edge4 perspective (Client)

In [1]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io

## Input filename that you want to clean data

In [2]:
# this pandas version dose not support directlly loading the file from url.
url = 'https://github.com/tawanmeow/privacy-sdn-logfile/tree/main/Experiment2'
filename = "/Users/kalika/PycharmProjects/Privacy_SDN_Edge_IoT/PlanB/Experiment2/test2_client.txt"
filename_csv = "/Users/kalika/PycharmProjects/Privacy_SDN_Edge_IoT/PlanB/Experiment2/test2_client.txt"


## Functions for getting the throughput data from dataframe df

In [3]:
def getDataframeThru(df,start_row,measurement_interval,header_range):
    '''
    This functions will import the data from txt file and return the dataframe without the header of txt file.
    Input: 
        measurement_interval = 30 (sec) : 
        header_range = 10 lines
        start_row = 0 
    Return: 
        df1t : dataframe of througput and jitter
    '''
    
    df1 = df.drop(labels=range(start_row, header_range), axis=0)
    df1t = df1.drop(labels=range(measurement_interval, len(df)), axis=0)
    
    return df1t

In [4]:
def getDatafromTxT(filename, headerrange):
    """
    Get dataframe from txt file:
    filename : xxx.txt
    headerrange : number of lines that needed to be removed. 
    
    return : df : datafame type
    """
    h = headerrange + 1
    skip_1 = list(range(0,h, 1))
    df = pd.read_csv(filename,
        skiprows=skip_1,
        header=None,
        delimiter=' ',
        skipinitialspace=True,
        error_bad_lines=False)
    return df

## Run functions:

In [6]:
headerrange = 14
df = getDatafromTxT(filename, headerrange)
print(df.head)

<bound method NDFrame.head of           0      1      2            3         4        5       6           7  \
0    edge5:      [     5]    0.00-1.00       sec     1.81  MBytes        15.2   
1    edge1:      [     5]    1.00-2.00       sec     1.75  MBytes        14.7   
2    edge5:      [     5]    1.00-2.00       sec     1.74  MBytes        14.6   
3    edge1:      [     5]    2.00-3.00       sec     1.85  MBytes        15.5   
4    edge5:      [     5]    2.00-3.00       sec     1.85  MBytes        15.5   
..      ...    ...    ...          ...       ...      ...     ...         ...   
709  edge1:  iperf  Done.          NaN       NaN      NaN     NaN         NaN   
710  edge5:      [     5]  29.00-30.00       sec     1.86  MBytes        15.6   
711  edge5:      [    ID]     Interval  Transfer  Bitrate  Jitter  Lost/Total   
712  edge5:    NaN    NaN          NaN       NaN      NaN     NaN         NaN   
713  edge5:  iperf  Done.          NaN       NaN      NaN     NaN         NaN  

b'Skipping line 74: expected 10 fields, saw 26\nSkipping line 76: expected 10 fields, saw 14\nSkipping line 77: expected 10 fields, saw 14\nSkipping line 81: expected 10 fields, saw 26\nSkipping line 83: expected 10 fields, saw 14\nSkipping line 84: expected 10 fields, saw 14\nSkipping line 87: expected 10 fields, saw 14\nSkipping line 94: expected 10 fields, saw 12\nSkipping line 95: expected 10 fields, saw 12\nSkipping line 157: expected 10 fields, saw 26\nSkipping line 159: expected 10 fields, saw 14\nSkipping line 160: expected 10 fields, saw 14\nSkipping line 164: expected 10 fields, saw 26\nSkipping line 166: expected 10 fields, saw 14\nSkipping line 167: expected 10 fields, saw 14\nSkipping line 170: expected 10 fields, saw 14\nSkipping line 176: expected 10 fields, saw 12\nSkipping line 180: expected 10 fields, saw 12\nSkipping line 239: expected 10 fields, saw 26\nSkipping line 241: expected 10 fields, saw 14\nSkipping line 242: expected 10 fields, saw 14\nSkipping line 247: e

In [7]:
## Find start row index of itteration
def getStartEndID(df,start_data,end_data):
    """
    to clean dataframe and return the data with new header
    Input: 
        df : datafram without header of txt file 
    Output
        strat_indices_list : start indices list
    """
    # creating and passing series to new column
    df["Start"]= df[3].str.find(start_data)
    df["End"]= df[3].str.find(end_data)

    index = df.index
    strat_indices = index[df["Start"]==0.0]
    strat_indices_list = strat_indices.tolist()
    end_indices = index[df["End"]==0.0]
    end_indices_list = end_indices.tolist()
    
    
    return strat_indices_list, end_indices_list
    

In [8]:
def getCleanData(df,strat_indices_list,end_indices_list):
    """
    
    """
    df_all = df.drop(labels=range(1, len(df)), axis=0) # create new df
    start_row = 0
    c = 0
    for i in strat_indices_list: 
        
        h = i
        print('h =',h)
        m = end_indices_list[c]
        print('m =', m)
        df1 = getDataframeThru(df,start_row,m,h)
        print('df1 = ', df1)
        result = pd.concat([df_all,df1])
        df_all = result
        c = c + 1
        if i == 0:
            
            df_all = df_all.drop(labels=0, axis=0)
        
        
    return df_all

## Run functions

In [13]:
start_data = '1.00-2.00'
end_data = '27.00-28.00'
strat_indices_list, end_indices_list = getStartEndID(df,start_data,end_data)
print('strat_indices_list: ', strat_indices_list)
print('end_indices_list: ', end_indices_list)


strat_indices_list:  [1, 2, 73, 74, 143, 147, 216, 220, 288, 292, 362, 363, 434, 435, 506, 507, 578, 579, 650, 651]
end_indices_list:  [53, 54, 125, 126, 196, 199, 269, 272, 341, 344, 414, 415, 486, 487, 558, 559, 630, 631, 702, 703]


In [14]:
df_all = getCleanData(df,strat_indices_list,end_indices_list)

h = 1
m = 53
df1 =           0  1   2            3    4     5       6     7          8       9  \
1   edge1:  [  5]    1.00-2.00  sec  1.75  MBytes  14.7  Mbits/sec  1270.0   
2   edge5:  [  5]    1.00-2.00  sec  1.74  MBytes  14.6  Mbits/sec  1260.0   
3   edge1:  [  5]    2.00-3.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
4   edge5:  [  5]    2.00-3.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
5   edge1:  [  5]    3.00-4.00  sec  1.88  MBytes  15.8  Mbits/sec  1360.0   
6   edge5:  [  5]    3.00-4.00  sec  1.89  MBytes  15.9  Mbits/sec  1370.0   
7   edge1:  [  5]    4.00-5.00  sec  1.86  MBytes  15.6  Mbits/sec  1350.0   
8   edge5:  [  5]    4.00-5.00  sec  1.86  MBytes  15.6  Mbits/sec  1350.0   
9   edge1:  [  5]    5.00-6.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
10  edge5:  [  5]    5.00-6.00  sec  1.86  MBytes  15.6  Mbits/sec  1350.0   
11  edge1:  [  5]    6.00-7.00  sec  1.91  MBytes  16.0  Mbits/sec  1380.0   
12  edge5:  [  5]    6.00-7.00  sec  1.89  M

558   -1.0  0.0  
h = 578
m = 630
df1 =            0  1   2            3    4     5       6     7          8       9  \
578  edge1:  [  5]    1.00-2.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
579  edge5:  [  5]    1.00-2.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
580  edge1:  [  5]    2.00-3.00  sec  1.84  MBytes  15.4  Mbits/sec  1330.0   
581  edge5:  [  5]    2.00-3.00  sec  1.82  MBytes  15.3  Mbits/sec  1320.0   
582  edge1:  [  5]    3.00-4.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
583  edge5:  [  5]    3.00-4.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
584  edge1:  [  5]    4.00-5.00  sec  1.80  MBytes  15.1  Mbits/sec  1300.0   
585  edge5:  [  5]    4.00-5.00  sec  1.80  MBytes  15.1  Mbits/sec  1300.0   
586  edge1:  [  5]    5.00-6.00  sec  1.82  MBytes  15.3  Mbits/sec  1320.0   
587  edge5:  [  5]    5.00-6.00  sec  1.82  MBytes  15.3  Mbits/sec  1320.0   
588  edge1:  [  5]    6.00-7.00  sec  1.85  MBytes  15.5  Mbits/sec  1340.0   
589  edge5:

In [17]:
# Replace new columns header

df_all_new = df_all.drop(df_all.columns[[1,2,4,6,8,10]], axis=1)  ## remove unnecessary columns
df_all_new.rename({0 : 'Edge', 3 :'Interval', 5 : 'Transfer', 7 :'Bitrate', 8 :'Jitter', 9 :'Lost/Total Datagrams'}, axis=1, inplace=True)
print(df_all_new)

       Edge     Interval Transfer Bitrate  Lost/Total Datagrams  End
0    edge5:    0.00-1.00     1.81    15.2                1310.0 -1.0
1    edge1:    1.00-2.00     1.75    14.7                1270.0 -1.0
2    edge5:    1.00-2.00     1.74    14.6                1260.0 -1.0
3    edge1:    2.00-3.00     1.85    15.5                1340.0 -1.0
4    edge5:    2.00-3.00     1.85    15.5                1340.0 -1.0
..      ...          ...      ...     ...                   ...  ...
698  edge1:  25.00-26.00     1.84    15.4                1330.0 -1.0
699  edge5:  25.00-26.00     1.85    15.5                1340.0 -1.0
700  edge1:  26.00-27.00     1.88    15.8                1360.0 -1.0
701  edge5:  26.00-27.00     1.86    15.6                1350.0 -1.0
702  edge1:  27.00-28.00     1.85    15.5                1340.0  0.0

[1044 rows x 6 columns]


## Plot dataframe

In [19]:
print(df_all_new['Bitrate'])

0      15.2
1      14.7
2      14.6
3      15.5
4      15.5
       ... 
698    15.4
699    15.5
700    15.8
701    15.6
702    15.5
Name: Bitrate, Length: 1044, dtype: object


In [18]:

df_all_new['Bitrate'] = df_all_new['Bitrate'].astype(float) 

ValueError: could not convert string to float: 'Datagrams'

In [ ]:
print(len(df_all_new.index))
time = np.array(range(len(df_all_new.index)))
print(time)

df_all_new['Time'] = time
print(df_all_new.head)
df_all_new['Time'] = df_all_new['Time'].astype(int) 


In [ ]:
## Throughput

In [ ]:
# avergae throughput 
sumThroughput = df_all_new['Bitrate'].sum()
avgSumThroughput = sumThroughput/len(time)
print('avg Throughput :', avgSumThroughput)
print('var Throughput :', df_all_new['Bitrate'].var())

In [ ]:
# Throughput 
ax = plt.gca()
df_all_new.plot(kind='line', x='Time', y='Bitrate', ax =ax)
plt.legend(["Throughput"]);
ax.set_xlabel("Time (sec)")
ax.set_ylabel("Throughput (Mbits/s)")
avg_thr = np.ones(len(df_all_new.index),dtype = float)*avgSumThroughput
s = pd.DataFrame(avg_thr, columns = ['AVG Throghput'])
s.plot(ax=ax)
plt.show()

## Jitter

In [ ]:
df_all_new['Jitter'] = df_all_new['Jitter'].astype(float) 

In [ ]:
# avergae jitter 
sumThroughput = df_all_new['Jitter'].sum()
avgSumThroughput = sumThroughput/len(time)
print('avg Jitter :', avgSumThroughput)
print('var Jitter :', df_all_new['Jitter'].var())

In [ ]:
avg_thr = np.ones(len(df_all_new.index),dtype = float)*avgSumThroughput
J = pd.DataFrame(avg_thr, columns = ['AVG Jitter'])

ax2 = plt.gca()
df_all_new.plot(kind='line', x='Time', y='Jitter', ax =ax2)
plt.legend(["Jitter"]);
ax2.set_xlabel("Time (sec)")
ax2.set_ylabel("Jitter (sec)")
J.plot(ax=ax2)
plt.show()